<a href="https://colab.research.google.com/github/Lmalviya/machineTranslationTask/blob/main/machineTranslationUsingLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00


In [43]:
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from torch.utils.data import Dataset

from datasets import load_dataset

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [38]:
class Dictionalry:
  def __init__(self, name):
    self.name = name
    self.word2index = {}
    self.word2count = {}
    self.index2word = {0: "SOS", 1: "EOS"}
    self.count = 2

  def addWords(self, sentence):
    wordList = sentence.split(" ")
    for word in wordList:
      word = Dictionalry.normalizeString(word)
      if word not in self.word2index.keys():
        self.word2index[self.count] = word
        self.index2word[word] = self.count
        self.word2count[word] = 1
        self.count += 1
      else:
        self.word2count[word] += 1

  def __name__(self):
    return self.name

  def __size__(self):
    return self.count-2

  def __wordFrequency__(self, word):
    return self.word2count[word]

  def __index__(self, word):
    return self.word2index[word]

  def __word__(self, index):
    return self.index2word[index]

  @staticmethod
  def normalizeString(word):
    word = word.lower().strip()
    return word


langEn = Dictionalry("english_dictionary")
langFr = Dictionalry("french_dictionary")

this is lakhan. i am from #9gnd9@


## Download dataset and build the vocab

In [25]:
books = load_dataset("opus_books", "en-fr")

Generating train split:   0%|          | 0/127085 [00:00<?, ? examples/s]

In [39]:
dataset_en_to_fr_list = []
for item in books['train']:
  tmpItem = {
      'english': item['translation']['en'],
      'french': item['translation']['fr']
  }
  dataset_en_to_fr_list.append(tmpItem)

dataset_df = pd.DataFrame(dataset_en_to_fr_list)
print(f'Number of data points: {len(dataset_df)}')
dataset_df.head()

Number of data points: 127085


,english,french
0,The Wanderer,Le grand Meaulnes
1,Alain-Fournier,Alain-Fournier
2,First Part,PREMIÈRE PARTIE
3,I,CHAPITRE PREMIER
4,THE BOARDER,LE PENSIONNAIRE


In [40]:
## Build the courpus
englishCorpus = ' '.join(dataset_df['english'].to_list())
frenchCorpus = ' '.join(dataset_df['french'].to_list())

## Build the dictionary
langEn.addWords(englishCorpus)
langFr.addWords(frenchCorpus)

In [41]:
print(f'English vocab size: {langEn.__size__()}')
print(f'French vocab size: {langFr.__size__()}')

English vocab size: 2715071
French vocab size: 2642300


## Build dataloader

In [44]:
class preprocessing(Dataset):
  def __init__(self, dataFrame, vocab_en, vocab_fr):
    super(preprocessing, self).__init__()
    self.data = dataFrame
    self.vocab_en = vocab_en
    self.vocab_fr = vocab_fr

  @staticmethod
  def word2index(text, vocab):
    word2indexList = []
    for word in text.split(" "):
      num = vocab.__index__(word)
      word2indexList.append(num)
    return word2indexList

  def __getitem__(self, index):
    langEnText = self.data.english[index]
    langFrText = self.data.french[index]
    word2IndexList_en = preprocessing.word2index(langEnText, self.vocab_en)
    word2IndexList_fr = preprocessing.word2index(langFrText, self.vocab_fr)
    return {
        "en_ids": word2IndexList_en,
        'fr_ids': word2IndexList_fr
    }

  def __len__(self):
    return len(self.data)


# Encoder-Decoder architecture

In [ ]:
class EncoderLSTM(nn.Module):
  def __init__(self, vocabSize, hiddenSize, embeddingSize, dropout_p=0.1):
    super(EncoderLSTM, self).__init__()

    self.embedding = nn.Embedding(vocabSize, embeddingSize)
    self.lstm = nn.LSTM(embeddingSize, hiddenSize, batch_first=True)
    self.dropout = nn.Dropout(dropout_p)

  def forward(self, inputs):
    embedding = self.embedding(inputs)
    embedding = self.dropout(embedding)
    output, hidden = self.LSTM(embedding)
    return output, hidden


In [45]:
SOS_TOKEN = 0
EOS_TOKEN = 1
MAX_LENGTH = 128

class DecoderLSTM(nn.Module):
  def __init__(self, vocabSize, hiddenSize, embeddingSize, dropout_p=0.1):
    super(DecoderLSTM, self).__init__()

    self.embedding = nn.Embedding(vocabSize, embeddingSize)
    self.lstm = nn.LSTM(embeddingSize, hiddenSize, batch_first=True)
    self.linear = nn.Linear(hiddenSize, vocabSize)
    self.dropout = nn.Dropout(dropout_p)

  def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
    batch_size = encoder_outputs.size(0)
    decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill('SOS_TOKEN')
    decoder_hidden = encoder_outputs
    decoder_output_list = []

    for i in range(MAX_LENGTH):
      decoder_output, decoder_hidden = self.forward_step(decoder_input, decoder_hidden)
      decoder_output_list.append(decoder_output)

      if target_tensor is not None:
        ## Teacher forcing: Feed the target as the next input
        decoder_input = target_tensor[:, i].unsqueeze(1)
      else:
        ## Without teacher forcing: use its own predictions as the next input
        _, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze(-1).detach()

    decoder_output = torch.cat(decoder_output_list, dim=1)
    decoder_output = F.log_softmax(decoder_output, dim=-1)
    return decoder_output, decoder_hidden, None # We return `None` for consistency in the training loop


  def forward_step(self, inputs, hidden):
    embedding = self.embedding(inputs)
    output, hidden = self.lstm(embedding)
    output = self.linear(output)
    return output, hidden

